# ELECCIÓN VARIANTE DEL MÉTODO *K-MEANS*
En este *notebooK* se analiza la capacidad del método *K-Means* para agrupar series temporales en función de sus patrones de demanda, considerando diferentes variantes del algoritmo. Para evaluar la calidad de la separación entre los grupos formados, se utiliza el coeficiente de silueta.

In [8]:
import warnings 
# Suppress all warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import seaborn as sns
import plotly.express as px
from tslearn.clustering import TimeSeriesKMeans, KShape, silhouette_score
import numpy as np
from tslearn.metrics import cdist_dtw
import time

In [9]:
datos = pd.read_csv('../../../1-DATOS/3-DATOS DE RESULTADOS/ANALISIS DESCRIPTIVO/medias_semanales_por_hora.csv', index_col=[0, 1], header=0)

In [10]:
%%time

# Escalar las series para estandarizar su media y varianza
scaler = TimeSeriesScalerMeanVariance()
datos_scaled = scaler.fit_transform(datos.T)  

# Método K-means, diferente métrica
km = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0)
labels_euclidean = km.fit_predict(datos_scaled)

km_dba = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=5, max_iter_barycenter=5, random_state=0)
labels_dtw = km_dba.fit_predict(datos_scaled)

km_sdtw = TimeSeriesKMeans(n_clusters=3, metric="softdtw", max_iter=5, max_iter_barycenter=5,
                           metric_params={"gamma": .5}, random_state=0)
labels_softdtw = km_sdtw.fit_predict(datos_scaled)

# Cálculo del coeficiente de silueta
print(silhouette_score(datos_scaled, labels_euclidean, metric="euclidean"))
print(silhouette_score(datos_scaled, labels_softdtw, metric="softdtw"))

# Cálculo de distancias DTW precomputadas para el coeficiente de silueta
distances_dtw = cdist_dtw(datos_scaled)
print(silhouette_score(distances_dtw, labels_dtw, metric="precomputed"))

0.3188196219609453
0.19877393289258227
0.14266623848173166
CPU times: total: 17min 1s
Wall time: 6min 6s
